## gTTS：文字轉語音

In [ ]:
!pip install gTTS 

In [ ]:
import gtts
import IPython.display as display

In [ ]:
print(gtts.lang.tts_langs())

In [ ]:
txt = 'gTTS可以透過線上翻譯，將文字轉換為語音，並將語音存檔'
tts = gtts.gTTS(text=txt, lang='zh-tw')
tts.save('gtts.mp3')
display.Audio("gtts.mp3", autoplay=True)

In [ ]:
f = open("gtts2.mp3", "wb")
tts1 = gtts.gTTS(text='gTTS可以透過線上翻譯，將文字轉換為語音，並將語音存檔', lang='zh-tw')
tts1.write_to_fp(f)
tts2 = gtts.gTTS(text='將文字分解為多個段落，分別轉換為語音', lang='zh-tw')
tts2.write_to_fp(f)
display.Audio("gtts2.mp3", autoplay=True)

## SpeechRecognition：語音轉文字(聲音檔)

In [ ]:
!pip install SpeechRecognition

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
with sr.WavFile("record1.wav") as source:  #讀取wav檔
    audio = r.record(source)
try:
    word = r.recognize_google(audio, language="zh-TW")
    print(word)
except:
    print("語音辨識失敗！")

## SpeechRecognition：語音轉文字(麥克風)

In [ ]:
!pip install pydub
!pip install ffmpeg

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
 
def record_audio(filename):
  js=Javascript("""
    async function recordAudio() {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = "開始錄音";
      capture.style.background = "orange";
      capture.style.color = "white";
      div.appendChild(capture);
      const stopCapture = document.createElement("button");
      stopCapture.textContent = "停止錄音";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      const audio = document.createElement('audio');
      const recordingVid = document.createElement("audio");
      audio.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({audio:true});
     
      let recorder = new MediaRecorder(stream);
      document.body.appendChild(div);
      div.appendChild(audio);
      audio.srcObject = stream;
      audio.muted = true;
      await audio.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => {capture.onclick = resolve; });
      recorder.start();
      capture.replaceWith(stopCapture);
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      stream.getAudioTracks()[0].stop();
      div.remove();
 
      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordAudio({})')
    binary=b64decode(data)
    with open(filename,"wb") as audio_file:
      audio_file.write(binary)
  except Exception as err:
    print(str(err))

record_audio("record.webm")
sound = AudioSegment.from_file("record.webm")
sound.export("record.wav", format ='wav')
r = sr.Recognizer()
with sr.WavFile("record.wav") as source:
    audio = r.record(source)
try:
    word = r.recognize_google(audio, language="zh-TW")
    print("語音辨識結果：\n" + word)
except:
    print("語音辨識失敗！")

## google_trans_new：文字翻譯

In [ ]:
!pip install google_trans_new

In [ ]:
!sed -i "s/response = (decoded_line + ']')/response = decoded_line/g" /usr/local/lib/python3.7/dist-packages/google_trans_new/google_trans_new.py

In [ ]:
from google_trans_new import google_translator
translator = google_translator()
text="今天天氣很好"
word = translator.translate(text, lang_src='zh-TW', lang_tgt='ja', pronounce=True)
print(word)

In [ ]:
from google_trans_new import google_translator
translator = google_translator()
print(translator.translate("今日の天気は良いです"))

In [ ]:
lang = translator.detect("今日の天気は良いです")
print(lang)

## 應用：AI智慧讀報機

In [ ]:
!pip install newspaper3k
!pip install gTTS
!pip install google_trans_new

In [ ]:
import newspaper
from newspaper import Article
from google_trans_new import google_translator
import gtts
import IPython.display as display
import random

paper = newspaper.build('http://cnn.com', language='en')
# paper = newspaper.build('http://www.cnbc.com', language='en')
# paper = newspaper.build('http://www.bbc.co.uk', language='en')
# paper = newspaper.build('http://www.foxnews.com', language='en')
urls = []
for article in paper.articles:
    url = article.url
    if '.html' in url:
        try:  #有時會產生無法擷取的錯誤,故使用try
          article = Article(url)
          article.download()
          article.parse()
          content = article.text
          if len(content)>0:
              urls.append(url)
              if len(urls)>10:
                break
        except: pass
if len(urls)>0:
    r = random.randint(0,len(urls)-1)
    url = urls[r]
    article = Article(url)
    article.download()
    article.parse()
    content = article.text
    if len(content)>5000: content = content[:4999]
    translator = google_translator()
    ret = translator.translate(content, lang_tgt='zh-TW')
    print(ret)
else: 
  ret = '無可用新聞！'
tts = gtts.gTTS(text=ret, lang='zh-tw')
tts.save('news.mp3')
display.Audio("news.mp3", autoplay=True)